In [1]:
#from config import CLIENT_ID, CLIENT_SECRET
CLIENT_ID = '65b864f672b34687acc2d1f1df57eb86'
CLIENT_SECRET = 'e051a779021e40c688863234ff4479e4'
import os
import pprint
import time
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
playlist_df = pd.read_csv(f'./Resources/playlists.csv')

In [3]:
playlist_df = playlist_df.dropna(axis=0)

In [4]:
playlist_df

,Playlist,Followers,Songs,Genre 1,Genre 2,URI
0,Today’s Top Hits,"31,728,611",50,Pop,R&B,spotify:playlist:37i9dQZF1DXcBWIGoYBM5M
1,Feel Good Indie Rock,"1,664,372",120,Rock,Indie,spotify:playlist:37i9dQZF1DX2sUQwD7tbmL
2,Indie Rock Road Trip,"895,126",100,Rock,Indie,spotify:playlist:37i9dQZF1DWUoqEG4WY6ce
3,Rock N' Run,"490,726",50,Rock,Workout,spotify:playlist:37i9dQZF1DWXx3Txis2L4x
4,Rock This,"4,649,112",50,Rock,Alternative,spotify:playlist:37i9dQZF1DXcF6B6QPhFDv
...,...,...,...,...,...,...
95,Beats & Rhymes,"403,154",100,Rap,K-Pop,spotify:playlist:37i9dQZF1DXcA6dRp8rwj6
96,The New Alt,"1,095,705",125,Alternative,Rock,spotify:playlist:37i9dQZF1DX82GYcclJ3Ug
97,Pop Punk's Not Dead,"523,146",80,Rock,Pop,spotify:playlist:37i9dQZF1DX1ewVhAJ17m4
98,Signed XOXO,"1,912,124",98,R&B,Pop,spotify:playlist:37i9dQZF1DX2A29LI7xHn1


In [5]:
# Set environment variables
os.environ['SPOTIPY_CLIENT_ID'] = CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = CLIENT_SECRET

In [6]:
# instantiate Spotify class
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [7]:
#playlist_dfs = []
songs_dfs = []

# Create counters.
record_count = 1
set_count = 1

# Iterate over playlist dataframe to grab song info for each playlist
for index, row in playlist_df[41:60].iterrows():
    
    print(f'Processing playlist {index} of {len(playlist_df)}')
    
    # Grabbing URI for the playlist
    playlist_uri = row['URI'] 
    print(playlist_uri)
    
    # Make temporary table for each playlist: subsetting playlist df
    temp = playlist_df[playlist_df['URI']==playlist_uri]
    
    # Get all songs from playlist
    song_count = len(sp.playlist_tracks(playlist_uri)["items"])
    
    # Temporary table for all songs in this playlist
    songs = []
    
    time.sleep(30)
    
    # Get info for each song
    for song_index in range(0, song_count):

        print(f'\t Adding info for song {song_index} out of {song_count}... \n')
        
        song_temp = temp.copy()

        
        try:
            ## Add song info to the table
            # Grab URI
            song_temp['Song URI'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["uri"]

            # Grab Popularity
            song_temp['Popularity'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["popularity"]

            # Grab Artist Name
            song_temp['Artist Name'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["artists"][0]['name']

            # Grab Song Name
            song_temp['Song Name'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["name"]

            # Grab popularity score
            songs.append(song_temp)

        except TypeError:
            print(f'\t Skipping info for song {song_index} out of {song_count}... \n')
            

    
    # Make song table
    songs = pd.concat(songs)
    
    # Add to playlist
    #playlist_dfs.append(songs)
    songs_dfs.append(songs)

# Bring everything together 
pd.concat(songs_dfs)
playlist_df = pd.DataFrame(pd.concat(songs_dfs))
#pd.concat(playlist_dfs)

Processing playlist 41 of 100
spotify:playlist:37i9dQZF1DXcFk5r8uS3l2
	 Adding info for song 0 out of 75... 

	 Adding info for song 1 out of 75... 

	 Adding info for song 2 out of 75... 

	 Adding info for song 3 out of 75... 

	 Adding info for song 4 out of 75... 

	 Adding info for song 5 out of 75... 

	 Adding info for song 6 out of 75... 

	 Adding info for song 7 out of 75... 

	 Adding info for song 8 out of 75... 

	 Adding info for song 9 out of 75... 

	 Adding info for song 10 out of 75... 

	 Adding info for song 11 out of 75... 

	 Adding info for song 12 out of 75... 

	 Adding info for song 13 out of 75... 

	 Adding info for song 14 out of 75... 

	 Adding info for song 15 out of 75... 

	 Adding info for song 16 out of 75... 

	 Adding info for song 17 out of 75... 

	 Adding info for song 18 out of 75... 

	 Adding info for song 19 out of 75... 

	 Adding info for song 20 out of 75... 

	 Adding info for song 21 out of 75... 

	 Adding info for song 22 out of 75...

	 Adding info for song 14 out of 100... 

	 Adding info for song 15 out of 100... 

	 Adding info for song 16 out of 100... 

	 Skipping info for song 16 out of 100... 

	 Adding info for song 17 out of 100... 

	 Adding info for song 18 out of 100... 

	 Adding info for song 19 out of 100... 

	 Adding info for song 20 out of 100... 

	 Adding info for song 21 out of 100... 

	 Adding info for song 22 out of 100... 

	 Adding info for song 23 out of 100... 

	 Adding info for song 24 out of 100... 

	 Adding info for song 25 out of 100... 

	 Adding info for song 26 out of 100... 

	 Adding info for song 27 out of 100... 

	 Adding info for song 28 out of 100... 

	 Adding info for song 29 out of 100... 

	 Adding info for song 30 out of 100... 

	 Adding info for song 31 out of 100... 

	 Adding info for song 32 out of 100... 

	 Adding info for song 33 out of 100... 

	 Adding info for song 34 out of 100... 

	 Adding info for song 35 out of 100... 

	 Adding info for song 36 out of

	 Adding info for song 38 out of 50... 

	 Adding info for song 39 out of 50... 

	 Adding info for song 40 out of 50... 

	 Adding info for song 41 out of 50... 

	 Adding info for song 42 out of 50... 

	 Adding info for song 43 out of 50... 

	 Adding info for song 44 out of 50... 

	 Adding info for song 45 out of 50... 

	 Adding info for song 46 out of 50... 

	 Adding info for song 47 out of 50... 

	 Adding info for song 48 out of 50... 

	 Adding info for song 49 out of 50... 

Processing playlist 46 of 100
spotify:playlist:37i9dQZF1DX6GwdWRQMQpq
	 Adding info for song 0 out of 50... 

	 Adding info for song 1 out of 50... 

	 Adding info for song 2 out of 50... 

	 Adding info for song 3 out of 50... 

	 Adding info for song 4 out of 50... 

	 Adding info for song 5 out of 50... 

	 Adding info for song 6 out of 50... 

	 Adding info for song 7 out of 50... 

	 Adding info for song 8 out of 50... 

	 Adding info for song 9 out of 50... 

	 Adding info for song 10 out of 50...

	 Adding info for song 23 out of 100... 

	 Adding info for song 24 out of 100... 

	 Adding info for song 25 out of 100... 

	 Adding info for song 26 out of 100... 

	 Adding info for song 27 out of 100... 

	 Adding info for song 28 out of 100... 

	 Adding info for song 29 out of 100... 

	 Adding info for song 30 out of 100... 

	 Adding info for song 31 out of 100... 

	 Adding info for song 32 out of 100... 

	 Adding info for song 33 out of 100... 

	 Adding info for song 34 out of 100... 

	 Adding info for song 35 out of 100... 

	 Adding info for song 36 out of 100... 

	 Adding info for song 37 out of 100... 

	 Adding info for song 38 out of 100... 

	 Adding info for song 39 out of 100... 

	 Adding info for song 40 out of 100... 

	 Adding info for song 41 out of 100... 

	 Adding info for song 42 out of 100... 

	 Adding info for song 43 out of 100... 

	 Adding info for song 44 out of 100... 

	 Adding info for song 45 out of 100... 

	 Adding info for song 46 out of 1

	 Adding info for song 16 out of 100... 

	 Adding info for song 17 out of 100... 

	 Adding info for song 18 out of 100... 

	 Adding info for song 19 out of 100... 

	 Adding info for song 20 out of 100... 

	 Adding info for song 21 out of 100... 

	 Adding info for song 22 out of 100... 

	 Adding info for song 23 out of 100... 

	 Adding info for song 24 out of 100... 

	 Adding info for song 25 out of 100... 

	 Adding info for song 26 out of 100... 

	 Adding info for song 27 out of 100... 

	 Adding info for song 28 out of 100... 

	 Adding info for song 29 out of 100... 

	 Adding info for song 30 out of 100... 

	 Adding info for song 31 out of 100... 

	 Adding info for song 32 out of 100... 

	 Adding info for song 33 out of 100... 

	 Adding info for song 34 out of 100... 

	 Adding info for song 35 out of 100... 

	 Adding info for song 36 out of 100... 

	 Adding info for song 37 out of 100... 

	 Adding info for song 38 out of 100... 

	 Adding info for song 39 out of 1

	 Adding info for song 19 out of 95... 

	 Adding info for song 20 out of 95... 

	 Adding info for song 21 out of 95... 

	 Adding info for song 22 out of 95... 

	 Adding info for song 23 out of 95... 

	 Adding info for song 24 out of 95... 

	 Adding info for song 25 out of 95... 

	 Adding info for song 26 out of 95... 

	 Adding info for song 27 out of 95... 

	 Adding info for song 28 out of 95... 

	 Adding info for song 29 out of 95... 

	 Adding info for song 30 out of 95... 

	 Adding info for song 31 out of 95... 

	 Adding info for song 32 out of 95... 

	 Adding info for song 33 out of 95... 

	 Adding info for song 34 out of 95... 

	 Adding info for song 35 out of 95... 

	 Adding info for song 36 out of 95... 

	 Adding info for song 37 out of 95... 

	 Adding info for song 38 out of 95... 

	 Adding info for song 39 out of 95... 

	 Adding info for song 40 out of 95... 

	 Adding info for song 41 out of 95... 

	 Adding info for song 42 out of 95... 

	 Adding info fo

	 Adding info for song 70 out of 100... 

	 Adding info for song 71 out of 100... 

	 Adding info for song 72 out of 100... 

	 Adding info for song 73 out of 100... 

	 Adding info for song 74 out of 100... 

	 Adding info for song 75 out of 100... 

	 Adding info for song 76 out of 100... 

	 Adding info for song 77 out of 100... 

	 Adding info for song 78 out of 100... 

	 Adding info for song 79 out of 100... 

	 Adding info for song 80 out of 100... 

	 Adding info for song 81 out of 100... 

	 Adding info for song 82 out of 100... 

	 Adding info for song 83 out of 100... 

	 Adding info for song 84 out of 100... 

	 Adding info for song 85 out of 100... 

	 Adding info for song 86 out of 100... 

	 Adding info for song 87 out of 100... 

	 Adding info for song 88 out of 100... 

	 Adding info for song 89 out of 100... 

	 Adding info for song 90 out of 100... 

	 Adding info for song 91 out of 100... 

	 Adding info for song 92 out of 100... 

	 Adding info for song 93 out of 1

	 Adding info for song 64 out of 80... 

	 Adding info for song 65 out of 80... 

	 Adding info for song 66 out of 80... 

	 Adding info for song 67 out of 80... 

	 Adding info for song 68 out of 80... 

	 Adding info for song 69 out of 80... 

	 Adding info for song 70 out of 80... 

	 Adding info for song 71 out of 80... 

	 Adding info for song 72 out of 80... 

	 Adding info for song 73 out of 80... 

	 Adding info for song 74 out of 80... 

	 Adding info for song 75 out of 80... 

	 Adding info for song 76 out of 80... 

	 Adding info for song 77 out of 80... 

	 Adding info for song 78 out of 80... 

	 Adding info for song 79 out of 80... 

Processing playlist 58 of 100
spotify:playlist:37i9dQZF1DX5wgKYQVRARv
	 Adding info for song 0 out of 100... 

	 Adding info for song 1 out of 100... 

	 Adding info for song 2 out of 100... 

	 Adding info for song 3 out of 100... 

	 Adding info for song 4 out of 100... 

	 Adding info for song 5 out of 100... 

	 Adding info for song 6 out

	 Adding info for song 77 out of 100... 

	 Adding info for song 78 out of 100... 

	 Adding info for song 79 out of 100... 

	 Adding info for song 80 out of 100... 

	 Adding info for song 81 out of 100... 

	 Adding info for song 82 out of 100... 

	 Adding info for song 83 out of 100... 

	 Adding info for song 84 out of 100... 

	 Adding info for song 85 out of 100... 

	 Adding info for song 86 out of 100... 

	 Adding info for song 87 out of 100... 

	 Adding info for song 88 out of 100... 

	 Adding info for song 89 out of 100... 

	 Adding info for song 90 out of 100... 

	 Adding info for song 91 out of 100... 

	 Adding info for song 92 out of 100... 

	 Adding info for song 93 out of 100... 

	 Adding info for song 94 out of 100... 

	 Adding info for song 95 out of 100... 

	 Adding info for song 96 out of 100... 

	 Adding info for song 97 out of 100... 

	 Adding info for song 98 out of 100... 

	 Adding info for song 99 out of 100... 



## maybe there is a better way to do this but the following line isn't something that we had and therefore the dataframe I had exported previously only grabbed the last "songs_dfs" and was returned as a list instead of a dataframe

In [8]:
playlist_df2 = pd.DataFrame(pd.concat(songs_dfs))

In [9]:
playlist_df

,Playlist,Followers,Songs,Genre 1,Genre 2,URI,Song URI,Popularity,Artist Name,Song Name
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:1KLyyQ9B4URptPsozR44l8,41,Amythyst Kiah,Chained To The Rhythm
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:1BIX2BQa6BroVxG6O97UWk,0,The War and Treaty,That's How Love Is Made
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:5MwOpeeCnMTXxTAtM9nsYr,47,Ben Harper,Need To Know Basis
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:47BdliCI9XP3HWl0CsR23A,27,Shemekia Copeland,Gullah Geechee
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:08VbstN22j4jlgMemYE2xk,11,Bad Flamingo,Bees Making Honey
...,...,...,...,...,...,...,...,...,...,...
59,Heavy Metal,"949,500",200,Metal,Rock,spotify:playlist:37i9dQZF1DX9qNs32fujYe,spotify:track:4G8EhnfKvBQVlOJ4xXCHpQ,56,Parkway Drive,Wild Eyes
59,Heavy Metal,"949,500",200,Metal,Rock,spotify:playlist:37i9dQZF1DX9qNs32fujYe,spotify:track:6EF0xhfKtQNqUPz2mnE5BD,56,Mastodon,The Motherload
59,Heavy Metal,"949,500",200,Metal,Rock,spotify:playlist:37i9dQZF1DX9qNs32fujYe,spotify:track:7M7AFtWYg73Kvj5cTZUjiZ,56,Beartooth,The Past Is Dead
59,Heavy Metal,"949,500",200,Metal,Rock,spotify:playlist:37i9dQZF1DX9qNs32fujYe,spotify:track:7JP8lWYrxpA8luKU8IPd3Y,56,Asking Alexandria,Alone Again


In [10]:
playlist_df.to_csv('Resources/41_60_playlists.csv')

In [11]:
playlist_df2

,Playlist,Followers,Songs,Genre 1,Genre 2,URI,Song URI,Popularity,Artist Name,Song Name
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:1KLyyQ9B4URptPsozR44l8,41,Amythyst Kiah,Chained To The Rhythm
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:1BIX2BQa6BroVxG6O97UWk,0,The War and Treaty,That's How Love Is Made
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:5MwOpeeCnMTXxTAtM9nsYr,47,Ben Harper,Need To Know Basis
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:47BdliCI9XP3HWl0CsR23A,27,Shemekia Copeland,Gullah Geechee
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:08VbstN22j4jlgMemYE2xk,11,Bad Flamingo,Bees Making Honey
...,...,...,...,...,...,...,...,...,...,...
59,Heavy Metal,"949,500",200,Metal,Rock,spotify:playlist:37i9dQZF1DX9qNs32fujYe,spotify:track:4G8EhnfKvBQVlOJ4xXCHpQ,56,Parkway Drive,Wild Eyes
59,Heavy Metal,"949,500",200,Metal,Rock,spotify:playlist:37i9dQZF1DX9qNs32fujYe,spotify:track:6EF0xhfKtQNqUPz2mnE5BD,56,Mastodon,The Motherload
59,Heavy Metal,"949,500",200,Metal,Rock,spotify:playlist:37i9dQZF1DX9qNs32fujYe,spotify:track:7M7AFtWYg73Kvj5cTZUjiZ,56,Beartooth,The Past Is Dead
59,Heavy Metal,"949,500",200,Metal,Rock,spotify:playlist:37i9dQZF1DX9qNs32fujYe,spotify:track:7JP8lWYrxpA8luKU8IPd3Y,56,Asking Alexandria,Alone Again


In [13]:
playlist_df_1_40 = pd.read_csv(f'./Resources/1_40_playlists.csv')

dfs = playlist_df, playlist_df_1_40
playlist_df_1_40

In [14]:
playlist_df1_60 = pd.DataFrame(pd.concat(dfs))
playlist_df1_60

,Playlist,Followers,Songs,Genre 1,Genre 2,URI,Song URI,Popularity,Artist Name,Song Name,Unnamed: 0.1,Unnamed: 0
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:1KLyyQ9B4URptPsozR44l8,41,Amythyst Kiah,Chained To The Rhythm,NaN,NaN
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:1BIX2BQa6BroVxG6O97UWk,0,The War and Treaty,That's How Love Is Made,NaN,NaN
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:5MwOpeeCnMTXxTAtM9nsYr,47,Ben Harper,Need To Know Basis,NaN,NaN
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:47BdliCI9XP3HWl0CsR23A,27,Shemekia Copeland,Gullah Geechee,NaN,NaN
41,Blues Roots,"414,002",75,Blues,Roots,spotify:playlist:37i9dQZF1DXcFk5r8uS3l2,spotify:track:08VbstN22j4jlgMemYE2xk,11,Bad Flamingo,Bees Making Honey,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3171,Funky Heavy Bluesy,"840,200",69,Blues,Rock,spotify:playlist:37i9dQZF1DX9stbPFTxeaB,spotify:track:6PJE3j017zKto7sv2wUrNy,31,Alvin Youngblood Hart,Motivational Speaker,39.0,NaN
3172,Funky Heavy Bluesy,"840,200",69,Blues,Rock,spotify:playlist:37i9dQZF1DX9stbPFTxeaB,spotify:track:0X4vLevfEeJIgjVpKedbgK,32,Ana Popovic,Lives That Don't Exist,39.0,NaN
3173,Funky Heavy Bluesy,"840,200",69,Blues,Rock,spotify:playlist:37i9dQZF1DX9stbPFTxeaB,spotify:track:4gvttoQ87qNDYrtVX5TlHG,28,Paulo Mendonca,Honey,39.0,NaN
3174,Funky Heavy Bluesy,"840,200",69,Blues,Rock,spotify:playlist:37i9dQZF1DX9stbPFTxeaB,spotify:track:2VaKZQ6d3qY0roo4E1Hapg,24,Bernard Allison,Groove With Me,39.0,NaN


In [15]:
playlist_df1_60.to_csv('Resources/1_60_playlists.csv')